In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler,FunctionTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from feature_engine.datetime import DatetimeFeatures
from collections import Counter

# Rótulo = CLASSI_FIN

In [3]:
df = pd.read_csv("dengue_sinan.csv")
df.head()

C:\Users\Danilo\AppData\Local\Temp\ipykernel_3032\1735460736.py:1: DtypeWarning: Columns (30,31,53,54,55,59,61,63,65,71,83,89,100,116,131,137) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dengue_sinan.csv")


,NU_NOTIFIC,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,...,DT_TRANSRM,DT_TRANSRS,DT_TRANSSE,NU_LOTE_V,NU_LOTE_H,CS_FLXRET,FLXRECEBI,IDENT_MICR,MIGRADO_W,ID_CNS_SUS_HASHED
0,158,2,A90,2016-03-05,201609,2016,29,292210,1381.0,2498731.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,4.0,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
1,298,2,A90,2016-02-15,201607,2016,29,293290,1385.0,3280969.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,4.0,NaN,ebc774c2fc9be73f8d403772dc14539dbe6e448e117f62...
2,5082,2,A90,2016-03-25,201612,2016,29,293250,1385.0,2800527.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,4.0,NaN,e0f037bc45eeea73df919230256055724a0bc45b990d62...
3,111262,2,A90,2016-03-24,201612,2016,29,291360,1385.0,2706628.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,4.0,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
4,166,2,A90,2016-03-13,201611,2016,29,292210,1381.0,2498731.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,4.0,NaN,595d3ab2cc2bfc76ee1e9ef090ac6abf0e7b64aee90e92...


In [4]:
df_backup = df

# Pré-Processamento

In [5]:
# df = df.dropna(axis=1, how='all') # Retirar colunas Com todos os dados vazios
df = df.loc[:, df.isnull().sum() < 0.5*df.shape[0]] # Retira colunas com mais de 50% dos dados vazios
df = df.dropna(subset=['CLASSI_FIN'])

df

,NU_NOTIFIC,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,...,HOSPITALIZ,CLASSI_FIN,CRITERIO,EVOLUCAO,DT_ENCERRA,TP_SISTEMA,DT_DIGITA,CS_FLXRET,IDENT_MICR,ID_CNS_SUS_HASHED
0,158,2,A90,2016-03-05,201609,2016,29,292210,1381.0,2498731.0,...,NaN,8.0,NaN,NaN,2016-05-05,2.0,2016-04-01,0.0,4.0,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
1,298,2,A90,2016-02-15,201607,2016,29,293290,1385.0,3280969.0,...,NaN,8.0,NaN,NaN,2016-04-16,2.0,2016-04-01,0.0,4.0,ebc774c2fc9be73f8d403772dc14539dbe6e448e117f62...
2,5082,2,A90,2016-03-25,201612,2016,29,293250,1385.0,2800527.0,...,2.0,10.0,2.0,1.0,2016-06-06,2.0,2016-04-01,1.0,4.0,e0f037bc45eeea73df919230256055724a0bc45b990d62...
3,111262,2,A90,2016-03-24,201612,2016,29,291360,1385.0,2706628.0,...,NaN,10.0,2.0,NaN,2016-03-31,2.0,2016-03-31,0.0,4.0,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
4,166,2,A90,2016-03-13,201611,2016,29,292210,1381.0,2498731.0,...,NaN,8.0,NaN,NaN,2016-05-13,2.0,2016-04-01,0.0,4.0,595d3ab2cc2bfc76ee1e9ef090ac6abf0e7b64aee90e92...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619929,29,2,A90,2024-01-24,202404,2024,29,292710,1393.0,3016234.0,...,NaN,8.0,NaN,NaN,2024-03-25,2.0,2024-02-01,0.0,4.0,f993338c091ea8b57a51cecaa9efe8e313598b01c831ac...
619967,1315882,2,A90,2024-01-11,202402,2024,29,290320,1403.0,9015728.0,...,NaN,8.0,NaN,NaN,2024-03-12,2.0,2024-01-18,0.0,4.0,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
620081,375452,2,A90,2024-02-05,202406,2024,29,292740,1380.0,614742.0,...,NaN,8.0,NaN,NaN,2024-04-06,2.0,2024-03-01,0.0,4.0,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
620100,52,2,A90,2024-01-30,202405,2024,29,293050,1381.0,3274713.0,...,NaN,8.0,NaN,NaN,2024-03-31,2.0,2024-02-05,0.0,4.0,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...


In [6]:
X,y = df.drop("CLASSI_FIN",axis=1), df["CLASSI_FIN"]

categorical_cols = X.select_dtypes(include=["number"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', SimpleImputer(strategy='most_frequent'), categorical_cols),
    ])


feature_selector = SelectKBest(score_func=chi2, k=5)

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # ('scaler', StandardScaler()),  # StandardScaler to normalize numerical data
    ('feature_selector', feature_selector),
    ('classifier', RandomForestClassifier())
])


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
pipeline.fit(X_train, y_train)

# Predict the results
y_pred = pipeline.predict(X_test)

# Print the selected features
selected_features = feature_selector.get_support(indices=True)
print("Selected features:", X.columns[selected_features])
# df.head()

Selected features: Index(['NU_NOTIFIC', 'ID_MUNICIP', 'CS_ESCOL_N', 'ACIDO_PEPT', 'AUTO_IMUNE'], dtype='object')
